<a href="https://colab.research.google.com/github/Tri334/Naive_Hoax_Covid/blob/master/filter_hoax_naive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Sastrawi

In [27]:
from bs4 import BeautifulSoup as sp
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import json
import pandas as pd

import base64
import requests

factory = StemmerFactory()
stemmer = factory.create_stemmer()

corpus_url = 'https://raw.githubusercontent.com/Tri334/Naive_Hoax_Covid/master/corpus.txt'
stop_url = 'https://raw.githubusercontent.com/Tri334/Naive_Hoax_Covid/master/stopword2016.txt'

req = requests.get(stop_url)
req = req.text
stopword = req.splitlines()

corpus = requests.get(corpus_url)
corpus = corpus.text

def openFile(corpus):
    file = open(corpus, encoding="utf8")
    soup = sp(file,'html.parser')
    doc_berita = soup.find_all("doc")
    return doc_berita

def priorProbability(berita):
    categori_only = {}
    prior_proba ={}
    jumlah = 0
    for item in berita:
        cat = item.find('cat').text
        cat = preprocessing(cat)
        if cat:
            if cat in categori_only:
                categori_only[cat]+=1
            else:
                categori_only[cat]=1
    for item in categori_only:
        jumlah+=categori_only[item]
    for items in categori_only:
        prior_proba[items]=(categori_only[items]/jumlah)
    return prior_proba

def preprocessing(berita):
    cleaning_words = re.sub("\W+"," ", berita)
    clean = cleaning_words.lower()

    tokenize = re.findall("\w+", clean)
    token = " ".join(tokenize)
    return stemmer.stem(token)

def webCraw(berita):
    list_berita = []
    list_berita_cat = []
    list_tidak_ada_cat = []
    for item in berita:
        berita = item.find("berita").text
        cat = item.find("cat").text
        if cat:
            cat = preprocessing(cat)
            berita = preprocessing(berita)
            list_berita.append(berita)
            list_berita_cat.append([berita] + [cat])
        else:
            list_tidak_ada_cat.append(berita)
    with open("list_berita.json", "w") as write_file:
        json.dump(list_berita, write_file)

    with open("list_berita_cat.json", "w") as write_file:
        json.dump(list_berita_cat, write_file)

    with open("list_tidak_ada_cat.json", "w") as write_file:
        json.dump(list_tidak_ada_cat, write_file)

def termUnik(list_berita,stopword):
    unique = []
    kata = " ".join(list_berita)
    token = kata.split(' ')
    for item in token:
        if item:
            if item not in stopword:
                if item not in unique:
                    unique.append(item)
    unique.sort()
    return unique

def dikategorikan(berita_cat):
    categorized = {}
    for item in berita_cat:
        if item[0] or item[1]:
            if item[1] in categorized:
                old = categorized[item[1]]
                categorized.update({item[1]:old +" "+item[0]})
            else:
                categorized[item[1]] = item[0]

    for key in categorized:
        tokenize = categorized[key].split(' ')
        categorized[key]= tokenize

    return categorized

def termWeight(categori,kata_unik):
    weight_cat_dict ={}
    for key in categori:
        # print(key)
        waight_temp = []
        tes = {}
        for i in range(len(kata_unik)):
            score = 0
            for item in categori[key]:
                if kata_unik[i] == item:
                    score += 1
            tes[kata_unik[i]] = score
            waight_temp.append(score)
        weight_cat_dict[key] = tes
    return weight_cat_dict

def conProba(weight_cat_dict,term_unik):
    sum_weight = {}
    possible = {}
    for item in weight_cat_dict:
        term_count = 0
        for val in weight_cat_dict[item]:
            term_count+=weight_cat_dict[item][val]
        sum_weight[item]= term_count

    for key in weight_cat_dict:
        poss_term=0
        temp = {}
        for value in weight_cat_dict[key]:
            poss_term = weight_cat_dict[key][value]
            p_kata = (poss_term+1)/(sum_weight[key]+ len(term_unik))
            temp[value]=p_kata
        possible[key]=temp
    return possible


In [28]:
list_berita = 'https://raw.githubusercontent.com/Tri334/Naive_Hoax_Covid/master/list_berita.json'
list_berita_cat = 'https://raw.githubusercontent.com/Tri334/Naive_Hoax_Covid/master/list_berita_cat.json'

list_berita = requests.get(list_berita)
list_berita = list_berita.text

list_berita_cat = requests.get(list_berita_cat)
list_berita_cat = list_berita_cat.text
